In [1]:
import pandas as pd
import os
from typing import List,Dict,Any

In [2]:
from langchain_core.documents import Document
from langchain_text_splitters import(
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter
)
print('Setup Completed')

Setup Completed


### Understanding Document Structure in Langchain

In [3]:
#create a simple Document
doc = Document(
    page_content = 'This is a test document',
    metadata = {'source': 'test.txt',
                'author':'Sandeep Vissa'}
)
print('Printing Document structure')
print(f'content: {doc.page_content}')
print(f'metadata: {doc.metadata}')

Printing Document structure
content: This is a test document
metadata: {'source': 'test.txt', 'author': 'Sandeep Vissa'}


### Reading Text(.txt) Files

In [4]:
#create simple txt files
import os
os.makedirs('data/text_files',exist_ok=True)

In [5]:
sample_texts={
    "data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}

for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)

print("✅ Sample text files created!")

✅ Sample text files created!


### Text Loader

In [11]:

from langchain.document_loaders import TextLoader
loader = TextLoader('data/text_files/python_intro.txt',encoding='utf-8')
documents = loader.load()
#print(type(documents))
#print(documents)
print(f'loaded {len(documents)} Document')
print(f'Content preview: {documents[0].page_content[:100]}')
print(f'Metadata: {documents[0].metadata}')

loaded 1 Document
Content preview: Python Programming Introduction

Python is a high-level, interpreted programming language known for 
Metadata: {'source': 'data/text_files/python_intro.txt'}
